In [1]:
import pandas as pd
import numpy as np
import surpyval as surv
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
from reliability.Distributions import Weibull_Distribution

In [2]:
#Nombrando columnas

colnames_afiliados = ['id_persona_afiliados', 'sexo', 'fecha_nacimiento', 'nivel_educacional', 'anos_educacion', 'estado_civil', 'comuna_domicilio', 'institucion_previsional'] 
colnames_cuentas = ['id_persona_cuentas', 'tipo_afiliacion', 'lugar_afiliacion', 'fecha_afiliacion', 'codigo_estado', 'modalidad_afiliacion', 'saldo_cuotas', 'saldo_cero']
colnames_rentas = ['id_persona_rentas', 'fecha_devengamiento_remuneracion', 'tipo_contrato', 'subsidio_incapacidad', 'actividad_economica', 'comuna_empleador', 'ingreso_imponible', 'n_trabajadores_empresa', 'renta_promedio_empresa', 'de_renta_promedio_empresa', 'ingreso_imponible_0', 'ingreso_imponible_tope', 'ingreso_minimo', 'id_empleador']  


#Importando Bases

afiliados3pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\1_afiliados.csv', names = colnames_afiliados, header = None, delimiter = ';')
cuentas3pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\2_cuentas.csv', names = colnames_cuentas, header = None, delimiter = ';')
rentas_imponibles3pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\5_rentas_imponibles.csv', names = colnames_rentas, header = None, delimiter = ';')

#afiliados5pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\5pp\1_afiliados.csv', names = colnames_afiliados, header = None, delimiter = ';')
#cuentas5pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\5pp\2_cuentas.csv', names = colnames_cuentas, header = None, delimiter = ';')
#rentas_imponibles5pp = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\5pp\5_rentas_imponibles.csv', names = colnames_rentas, header = None, delimiter = ';')

In [5]:
cuentas3pp["id_persona_cuentas"] = cuentas3pp["id_persona_cuentas"].astype(object)


'\n\n'

In [7]:
cols_keep = ['id_persona_afiliados', 'sexo', 'fecha_nacimiento', 'fecha_devengamiento_remuneracion', 'ingreso_imponible', 'ingreso_imponible_tope']


#Merge 3pp

afiliados_cuentas3pp = pd.merge(afiliados3pp, cuentas3pp, how='inner', left_on=['id_persona_afiliados'], right_on=['id_persona_cuentas'])
bdsc3pp = pd.merge(rentas_imponibles3pp, afiliados_cuentas3pp, how='inner', left_on=['id_persona_rentas'], right_on=['id_persona_afiliados'])

bdsc3pp = bdsc3pp[cols_keep]

bdsc3pp = bdsc3pp.query('fecha_devengamiento_remuneracion > 202006 & fecha_devengamiento_remuneracion < 202101')



bdsc3pp = bdsc3pp.rename(columns={'ingreso_imponible': 'remp_imp_t', 'ingreso_imponible_tope': 'rem_tope', 'id_persona_afiliados': 'id_pers'})

df_bdsc = bdsc3pp

df_bdsc.to_stata('df_bdsc_borrador.dta')

#Merge 5pp

#afiliados_cuentas5pp = pd.merge(afiliados5pp, cuentas5pp, how='inner', left_on=['id_persona_afiliados'], right_on=['id_persona_cuentas'])
#bdsc5pp = pd.merge(rentas_imponibles5pp, afiliados_cuentas5pp, how='inner', left_on=['id_persona_rentas'], right_on=['id_persona_afiliados'])

#bdsc5pp = bdsc5pp[cols_keep]

#bdsc5pp = bdsc5pp.query('fecha_devengamiento_remuneracion > 202002')


#Concatenar bases

#frames = [bdsc3pp, bdsc5pp]

#df_bdsc = pd.concat(frames)

#df_bdsc = df_bdsc.rename(columns={'ingreso_imponible': 'remp_imp_t', 'ingreso_imponible_tope': 'rem_tope', 'id_persona_afiliados': 'id_pers'})

In [13]:
afiliados3pp


#print(afiliados3pp.id_persona_afiliados)

#print(cuentas3pp.id_persona_cuentas)

,id_persona_afiliados,sexo,fecha_nacimiento,nivel_educacional,anos_educacion,estado_civil,comuna_domicilio,institucion_previsional
5,F,194001,5,4,99,7301,1003,C
74232773,M,190801,5,4,99,6115,1034,E
90783569,M,192807,1,4,2,13110,1008,C
74248006,M,191209,5,4,99,13110,1033,C
57712020,M,194408,5,4,99,13401,1005,C
...,...,...,...,...,...,...,...,...
94053212,X,0,99,99,99,0,9999,NaN
27872207,X,0,99,99,99,0,9999,NaN
11326360,X,0,99,99,99,0,9999,NaN
61321904,F,198208,5,4,99,13123,9999,NaN


In [ ]:
import pandas as pd
import numpy as np
import surpyval as surv
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
from reliability.Distributions import Weibull_Distribution

In [ ]:
df_bdsc = pd.read_stata(r'C:\Users\alfre\Python\BDSC\df_bdsc_borrador_procesado.dta')
df_bdsc = df_bdsc.rename(columns={'rent_imp_t': 'remuneracion_parcial'})
df_bdsc = df_bdsc.rename(columns={'rent_imp_tt': 'rent_imp_t'})


df_bdsc['ano'] = round(df_bdsc.fecha_devengamiento_remuneracion / 100)

In [ ]:
#Botando sexo y fecha nacimiento nan's

df_bdsc = df_bdsc.dropna(subset = ['fecha_nacimiento', 'sexo'])

#df_bdsc.nlargest(30, ['remp_imp_t'])
#df_bdsc.to_stata('muestra_Benjamin.dta')

print('No se estan cambiando los nombres remp_imp_t')

In [ ]:
###Cargando ESI y seleccionando no cotizantes

cols_keep_esi20 = ['ing_t_t', 'sexo', 'edad', 'fact_cal_esi', 'e2']
cols_keep_esi18 = ['ing_t_t', 'sexo', 'edad', 'fact_per', 'e2']


df_esi20 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2020---personas.csv')
df_esi20_nc = df_esi20.query('a1 == 2 or b5 == 1 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2') #Seleccionando no cotizantes
df_esi20_nc = df_esi20_nc[cols_keep_esi20] #Conservando solo columnas relevantes
df_esi20_nc['ano'] = 2020

df_esi19 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2019---personas.csv')
df_esi19_nc = df_esi19.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi19_nc = df_esi19_nc[cols_keep_esi20]
df_esi19_nc['ano'] = 2019

df_esi18 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2018---personas.csv')
df_esi18_nc = df_esi18.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi18_nc = df_esi18_nc[cols_keep_esi18]
df_esi18_nc = df_esi18_nc.rename(columns={'fact_per': 'fact_cal_esi'}) #renombrando columna factor expansión
df_esi18_nc['ano'] = 2018


df_esi17 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2017---personas.csv')
df_esi17_nc = df_esi17.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi17_nc = df_esi17_nc[cols_keep_esi18]
df_esi17_nc = df_esi17_nc.rename(columns={'fact_per': 'fact_cal_esi'}) #renombrando columna factor expansión
df_esi17_nc['ano'] = 2017

frames_esi = [df_esi20_nc, df_esi19_nc, df_esi18_nc, df_esi17_nc]
df_esi = pd.concat(frames_esi, ignore_index = True)


#Creando grupos etarios esi
df_esi['grupo_etario'] = 0

df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=17) & (df_esi['edad'] >= 15) , 1, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=25) & (df_esi['edad'] >= 18) , 2, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=45) & (df_esi['edad'] >= 26) , 3, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] >= 46) , 4, df_esi['grupo_etario']
   )

In [ ]:
#Definición de funciones

#Imputación hotdeck

def hotdeck_imputation(row):
    if pd.isna(row.remp_imp_t):
        fila_elegida = df_esi.query('sexo == @row.sexo').query('grupo_etario == @row.grupo_etario').query('ano == @row.ano').sample(n=1, weights = df_esi.fact_cal_esi).iloc[0] #
        row.remp_imp_t = fila_elegida.ing_t_t
        row['busc_trab'] = fila_elegida.e2
        row['imputado'] = True
    else:
        row['imputado'] = False
    return row

#Función Gini

def gini(array):
    #Se pasa a numpy
    array = array.to_numpy()
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))


#Ratio interdecil

def r80_20(series):
    top20, bottom20 = series.quantile([.8, .2])
    r = top20/bottom20
    return r

def r90_10(series):
    top10, bottom10 = series.quantile([.9, .1])
    r = top10/bottom10
    return r


#### Hotdeck + Weibull

def tope_en_hotdeck(row):
    if (row.rem_tope) == 1:
        valor_imputado = df_topes_imputados.query('rem_tope == 1').sample(n=1).iloc[0].remp_imp_t
        row.remp_imp_t = valor_imputado
        row['imputado_tope'] = True
    else:
        row['imputado_tope'] = False
    return row

In [ ]:
#Creando edad y grupos etarios

df_bdsc["edad"] = df_bdsc.fecha_devengamiento_remuneracion- df_bdsc.fecha_nacimiento

df_bdsc["edad"] = round(df_bdsc.edad / 100, ndigits = None)

df_bdsc['grupo_etario'] = 0


df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=17) & (df_bdsc['edad'] >= 15) , 1, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=25) & (df_bdsc['edad'] >= 18) , 2, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=45) & (df_bdsc['edad'] >= 26) , 3, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] >= 46) , 4, df_bdsc['grupo_etario']
   )

df_bdsc = df_bdsc.query('grupo_etario != 0')
#df_bdsc["edad"] = str(df_bdsc["edad"][0:1])

In [ ]:
print(df_bdsc.rem_tope.value_counts())
print(df_bdsc.remp_imp_t.isnull().sum())

In [ ]:
#Limpiando
    
df_bdsc = df_bdsc[df_bdsc.fecha_nacimiento != 0] #Drop nacimientos en 0
df_bdsc = df_bdsc[df_bdsc.sexo != "X"] #Drop sexo no identificado
df_bdsc = df_bdsc.query('remp_imp_t != 0') #Sacamos cotizaciones por $0

df_bdsc_dropna = df_bdsc.dropna(subset = ['remp_imp_t'])

In [ ]:
print(df_bdsc.sexo.value_counts())
print(df_bdsc.grupo_etario.value_counts())
print(df_esi.grupo_etario.value_counts())

In [ ]:
#Reemplazando lagunas por ceros

#df_bdsc_nanacero = df_bdsc

#df_bdsc_nanacero['remp_imp_t'] = np.where(
   #(pd.isna(df_bdsc_nanacero.remp_imp_t))  , 0, df_bdsc['remp_imp_t']
   #)

#df_bdsc_nanacero.to_stata('df_bdsc_nanacero.dta')

In [ ]:
df_bdsc_valor_remtope = df_bdsc_dropna.query('rem_tope == 1')

df_bdsc_valor_remtope =  df_bdsc_valor_remtope.groupby('fecha_devengamiento_remuneracion')['remp_imp_t'].agg(pd.Series.mode).reset_index()

df_bdsc_valor_remtope

In [ ]:
#Imputación de valores con tope imponible

#Weibull mensual
df_topes_imputados = pd.DataFrame()

for mes, group in df_bdsc_dropna.groupby('fecha_devengamiento_remuneracion'):
    censored_monthly_tot = group['rem_tope'].sum()#Cantidad de censurados
    censored_monthly = group.rem_tope #Censurados
    #cut_off_monthly = 3850000
    cut_off_monthly = df_bdsc_valor_remtope.query('fecha_devengamiento_remuneracion =='+ str(mes)).remp_imp_t
    #cut_off_monthly = group['rem_tope == 1'].agg(pd.Series.mode)
    print(cut_off_monthly)
    model = surv.Weibull.fit(group.remp_imp_t, censored_monthly)
    dist_monthly = Weibull_Distribution(alpha = model.params[0], beta = model.params[1])

    for i in range(len(group)):
        if group['rem_tope'].iloc[i] == 1:
            imp_random = dist_monthly.random_samples(1)
            while float(imp_random) < float(cut_off_monthly):
                imp_random = dist_monthly.random_samples(1)
            group.remp_imp_t.iloc[i] = imp_random
    print(mes)
    df_i = pd.DataFrame(group)
    df_topes_imputados = pd.concat([df_topes_imputados, df_i])


#df_topes_imputados nace de df_bdsc_dropna

In [ ]:
#df_topes_imputados.to_stata('df_topes_imputados_2020.dta')

In [ ]:
#Aplicando Hotdeck

tqdm.pandas(desc="my bar!")

df_bdsc_imputadahd = df_bdsc.progress_apply(hotdeck_imputation, axis = 1)




In [ ]:
df_bdsc_imputadahd = df_bdsc_imputadahd.drop(df_bdsc_imputadahd[(df_bdsc_imputadahd.remp_imp_t == 0) & (df_bdsc_imputadahd.busc_trab == 2)].index)

In [ ]:
df_hotdeck_tope = df_bdsc_imputadahd.progress_apply(tope_en_hotdeck, axis = 1)

In [ ]:
#Resultados solo cotizantes, sin imputación tope

#Gini solo cotizantes, sin imputación de tope

data = []

for fecha_devengamiento_remuneracion, ae_df in df_bdsc_dropna.groupby('fecha_devengamiento_remuneracion'):
    #display(actividad_economica)
    #display(gini(ae_df.ingreso_imponible))
    row = [
        fecha_devengamiento_remuneracion, 
        round(gini(ae_df.remp_imp_t), ndigits = 3), 
        round(r80_20(ae_df.remp_imp_t), ndigits = 3),
        round(r90_10(ae_df.remp_imp_t), ndigits = 3),
    ]
    data.append(row)
    
col_header = ['Mes', 'Gini', 'P80/P20', 'P90/10']

tabla1 = pd.DataFrame(data, columns = col_header)


tabla1

In [ ]:
#Gini solo cotizantes, con imputación de tope

data = []

for fecha_devengamiento_remuneracion, ae_df in df_topes_imputados.groupby('fecha_devengamiento_remuneracion'):
    #display(actividad_economica)
    #display(gini(ae_df.ingreso_imponible))
    row = [
        fecha_devengamiento_remuneracion, 
        round(gini(ae_df.remp_imp_t), ndigits = 3), 
        round(r80_20(ae_df.remp_imp_t), ndigits = 3),
        round(r90_10(ae_df.remp_imp_t), ndigits = 3), 
    ]
    data.append(row)
    
col_header = ['Mes', 'Gini', 'P80/P20', 'P90/10']

tabla1 = pd.DataFrame(data, columns = col_header)


tabla1

In [ ]:
#Gini imputación topes y no cotizantes

data = []

for fecha_devengamiento_remuneracion, ae_df in df_hotdeck_tope.groupby('fecha_devengamiento_remuneracion'):
    #display(actividad_economica)
    #display(gini(ae_df.ingreso_imponible))
    row = [
        fecha_devengamiento_remuneracion, 
        round(gini(ae_df.remp_imp_t), ndigits = 3), 
        round(r80_20(ae_df.remp_imp_t), ndigits = 3),
        round(r90_10(ae_df.remp_imp_t), ndigits = 3),
    ]
    data.append(row)
    
col_header = ['Mes', 'Gini', 'P80/P20', 'P90/10']

tabla1 = pd.DataFrame(data, columns = col_header)


tabla1

In [ ]:
df_bdsc_dropna.to_stata('df_bdsc_dropna.dta')

df_topes_imputados.to_stata('df_topes_imputados.dta')